In [57]:
!pip install langgraph langsmith

In [58]:
!pip install langchain_community langchain langchain_groq

In [59]:
from google.colab import userdata
groq_api_key = userdata.get('groq_api_key')
print(groq_api_key)

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


In [60]:
langsmith=userdata.get('LANGSMITH_API_KEY')
print(langsmith)

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


In [61]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Courselangraph"
os.environ["LANGCHAIN_API_KEY"] = langsmith

In [62]:
from langchain_groq import ChatGroq

In [63]:
llm = ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It", verbose=True)

# Start Building Chatbot using Langgraph

In [64]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages

In [65]:

class State(TypedDict):
  messages:Annotated[list[str],add_messages]

# Define input and output for the state graph
input = ...  # Replace with your input definition
output = ...  # Replace with your output definition

graph_builder = StateGraph(input=input, output=output)

In [66]:
graph_builder

In [67]:
def chatbot(state:State):
  return {"messages":llm.invoke(state['messages'])}

In [68]:
graph_builder.add_node("chatbot",chatbot)

In [69]:
graph_builder

In [70]:
graph_builder.add_edge(START,"chatbot")
graph_builder.add_edge("chatbot",END)

In [71]:
graph=graph_builder.compile()

In [72]:
from IPython.display import Image, display
try:
  display(Image(graph_builder.get_graph().draw_mermaid_png()))
except Exception:
  pass

In [74]:
# Restore the original input() function
del input

while True:
  user_input = input("User : ")  # Now this should work
  if user_input.lower() in ["quit","q"]:
    print("Good Bye")
    break
  for event in graph.stream({'messages':("user",user_input)}):
    print(event.values())
    for value in event.values():
      print(value['messages'])
      print("Assistant:",value['messages'].content)

User : Hi


BadRequestError: Error code: 400 - {'error': {'message': "'messages' : minimum number of items is 1", 'type': 'invalid_request_error'}}